In this notebook, we'll explore a sequence-to-sequence transformer model which we will train to translate language. Specifically, we will train the model to translate italian sentences to english by using a dataset from tatoeba. But you can of course train the model to translate it to any other language. 

The model that we are going to build is based on the famous paper "Attention is all you need. Before we dive into the model, we are first going to set our project up and do a bit of data processing.

### Setup

In [1]:
import torch 
torch.__version__

'2.1.2+cu121'

In [31]:
# requirements 
from __future__ import unicode_literals, print_function, division
from io import open 
import unicodedata
import re 
import random 
import math

import torch 
import torch.nn as nn 
from torch.nn import Transformer
from torch import Tensor
import torch.nn.functional as F 

import numpy as np 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from typing import List, Tuple

### Data processing

In [36]:
# Remove CC lines
# Initialize an empty list to store the processed lines
def removeCC(filepath, save_name):
    processed_lines = []

    # Open the file and read line by line
    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            # Find the index of "CC-BY"
            index = line.find("CC-BY")
            
            # If "CC-BY" is found in the line
            if index != -1:
                # Remove "CC-BY" and everything after it and add the processed line to the list
                processed_lines.append(line[:index].strip())
            else:
                # If "CC-BY" is not found, add the original line to the list
                processed_lines.append(line.strip())

    # Join the processed lines into a single string
    processed_text = "\n".join(processed_lines)

    # Write the processed text to a new file
    with open(f"./data/{save_name}", "w", encoding="utf-8") as file:
        file.write(processed_text)

In [37]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", " \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [38]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [39]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s-cleaned.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [40]:
# Only include sentences with max 10 words for quick testing.
# Increase for more complexity
MAX_LENGTH = 5

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p): 
    return len(p[0].split(" ")) < MAX_LENGTH and \
        len(p[1].split(" ")) < MAX_LENGTH # and \
        # p[1].startswith(eng_prefixes)
    
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [41]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    # pairs = filterPairs(pairs)
    # print("Trimmed to %s sentence pairs" % len(pairs))
    # print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("\nCounted words:")
    print(f"{input_lang.name} with {input_lang.n_words} words")
    print(f"{output_lang.name} with {output_lang.n_words} words")
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'it', True)
print("\nExample:")
print(random.choice(pairs))

Reading lines...
Read 374932 sentence pairs

Counted words:
it with 27279 words
en with 13628 words

Example:
['perche ero cosi stupido', 'why was i so stupid']


In [42]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

SRC_LANGUAGE = 'it'
TGT_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='it_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

c:\Users\Leo\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [43]:
example_sentence = "questa è una frase di esempio"
print(token_transform[SRC_LANGUAGE](example_sentence))

['questa', 'è', 'una', 'frase', 'di', 'esempio']


In [44]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(
        yield_tokens(pairs, ln),
        min_freq=1,
        specials=special_symbols,
        special_first=True
    )

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

### Setting up the model

![The transformer architecture](transformer-architecture.png "Transformer architecture")

First we start with the positional encoding. The positional encoding is applied to the input embeddings for both the encoder and decoder of the model. The positional encoding is very important because without it, the model would have no idea about the position of the input tokens. And that's a problem because transformer models process the sequence in parrallel. 

"That makes sense, but why don't we just feed in the position via the index into the model?", you may say now. A valid question. That is because longer sequences can grow quite large in magnitude, while values normalized between 0 and 1 would normalize differently for varying sequence lengths. 

Above we have the original equation of the positional encoding. Let's break it down: 

- The sin function is applied to every even element in the position, while the cos function is applied to every uneven element.
- pos is simply the position of the item in the sequence
- den is a range of the embedding size (for example 256) times 10000 (set by the authors of the paper) divided by the embedding size

In [45]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))

        # Apply sin and cos to the pos embedding to ensure no embeddings are the same
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        output = self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])
        print(output.shape)
        return output

Next, we also need a class of token embeddings. This is really just a big lookup list of the vocabulary size times the embedding size. This is to ensure that every token gets a unique tensor. 

In [ ]:
# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

And that's already it for the "helper functions" of our seq2seq transformer. Now we can get down to actually setting up the transformer model. While it can be intimidating at first, the structure of the transformer is actually quite simple (at least compared to some other model architectures in my opinion, for example LSTMs). Also, PyTorch offers a lot of the building blocks that we need. 

There are some hyperparameters that we need to set for our model: 
- Embedding size: 
- Number of attention heads: Attention is one of the key components of our model, as it can be used to capture the connections between the words in our texts. If we want to oversimplify it, attention is a bunch of matrix multiplication performed on top of our input embeddings (the key, query and value). k, q and v are obtained by multiplying the input embeddings with three different weight matrices that are being created during the training. The attention is randomly initiated, and more attention heads mean we can capture more relations of our words. That being said, too many attention heads can of course also lead to worse results.
- Source and target vocab size: Unique words in our source and target language. We need this to set up our token embeddings. 
- Feedforward dimension: The output of the attention heads is fed into a standard MLP neural network. This hyperparameter sets the size of that MLP. 
- Dropout: To avoid overfitting, we can randomly turn off parts of the model to force it to also use different "routes" within the model. If this is set to 0.1, then 10 % of the models neurons will be turned off. 

In [46]:
# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [47]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [48]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 256
NHEAD = 4
FFN_HID_DIM = 512
BATCH_SIZE = 256
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [49]:
#transformer.load_state_dict(torch.load("weights/transformer-translation-it-en-epoch-50.pt"))

In [50]:
from torchinfo import summary

print(summary(transformer, verbose=0))

Layer (type:depth-idx)                                                 Param #
Seq2SeqTransformer                                                     --
├─Transformer: 1-1                                                     --
│    └─TransformerEncoder: 2-1                                         --
│    │    └─ModuleList: 3-1                                            1,581,312
│    │    └─LayerNorm: 3-2                                             512
│    └─TransformerDecoder: 2-2                                         --
│    │    └─ModuleList: 3-3                                            2,372,352
│    │    └─LayerNorm: 3-4                                             512
├─Linear: 1-2                                                          3,502,910
├─TokenEmbedding: 1-3                                                  --
│    └─Embedding: 2-3                                                  6,984,448
├─TokenEmbedding: 1-4                                                  --
│  

In [51]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [52]:
def train_epoch(model, optimizer, train_pairs):
    model.train()
    losses = 0
    train_iter = train_pairs
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    
    for i, (src, tgt) in enumerate(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

        if (i+1) % 100 == 0:
            print(f"Batch {i+1}/{len(train_dataloader)} | Loss: {loss}")

    return losses / len(list(train_dataloader))


def evaluate(model, val_pairs):
    model.eval()
    losses = 0

    val_iter = val_pairs
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [53]:
torch.cuda.empty_cache()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_pairs, val_pairs = train_test_split(pairs, test_size=0.2, random_state=42)
train_pairs[:3], val_pairs[:3]

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 50

loss_history = {"train": [], "val": []}

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_pairs=train_pairs)
    end_time = timer()
    val_loss = evaluate(transformer, val_pairs=val_pairs)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    loss_history["train"].append(train_loss)
    loss_history["val"].append(val_loss)
    if epoch % 25 == 0:
        torch.save(transformer.state_dict(), f'./weights/transformer-translation-it-en-mini-epoch-{epoch}.pt')

: 

In [18]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [21]:
torch.save(transformer.state_dict(), './weights/transformer-translation-it-en-epoch-15.pt')

In [22]:
print(translate(transformer, "io sono un ragazzo"))

 i m not sure that tom will be able to
